# How these files are generated...

I use [readermode premium](https://readermode.io/) to covert text. it seems to do better than firefox reader mode....

Then I use this [hacked up chrome extension](https://github.com/davidbstein/chrome-page-downloader) to download everything into a json ball.

In [1]:
cd ..

/Users/stein/repos/privacy-code-tools


In [127]:
from coder.api import models
import nltk.data
import html2text
import bs4
import os
import json
from coder import settings
from coder.api import models
import datetime
import re
from collections import defaultdict
import datetime

In [54]:
DL_DIR=settings.env("DOWNLOADS_FOLDER")
TOKENIZER = nltk.data.load('tokenizers/punkt/english.pickle')

In [106]:
def to_coding_doc(html):
    converter = html2text.HTML2Text()
    converter.protect_links=True
    converter.single_line_break=False
    converter.inline_links=False
    converter.body_width=999999999999
    converter.strong_mark="__"
    paragraphs = converter.handle(html).split("\n\n")
    return list(map(_clean_paragraph, paragraphs))
    
def _clean_paragraph(paragraph):
    if paragraph.startswith("#"):
        match = re.match(r'(^#*)(.*)', paragraph)
        level = len(match[1])
        content = match[2].strip()
        return {
            "type": "section",
            "level": level,
            "content": TOKENIZER.tokenize(content),
        }
    elif paragraph.startswith(' ') and (paragraph.strip().startswith('* ') or paragraph.strip().startswith('1. ')):
        list_items = [
            re.match(r'(?P<space>^ +)(?P<bullet>.+?) (?P<content>.*)', raw_list_item).groupdict() 
            for raw_list_item in paragraph.split('\n')
        ]
        return {
            "type": "list",
            "content": [
                {
                    "depth": len(list_item['space'])//2,
                    "bullet": list_item['bullet'],
                    "content": TOKENIZER.tokenize(list_item['content'])
                } for list_item in list_items]
        }
    else:
        return {
            "type": "text",
            "content": TOKENIZER.tokenize(paragraph)
        }

In [156]:
# this is terrible and needs to be rewritten.
filenames = [f for f in os.listdir(dl_dir) if f.startswith("pp_snapshot")]
parsed_docs = defaultdict(dict)
for fn in filenames:
    with open(f'{DL_DIR}/{fn}') as f:
        content = json.load(f)
        host= content['document']['location']['hostname']
        parsed_docs[host][fn[-len("yyyy-mm-ddThh_mm_ss.mmmZ.json"):-len('.json')]] = {
            "raw_content": content,
            "parsed_content": to_coding_doc(content['cleaned_html']),
        }

In [157]:
def create_policy_instance_content(date_to_content_list):
    """
    takes the parsed content of the JSON from the scape and turns it into the content object for the document scanner
    """
    to_ret_parsed = []
    to_ret_raw = []
    for i, (fname, content) in enumerate(sorted(date_to_content_list)):
        parsed_content = content['parsed_content']
        raw_content = content['raw_content']
        ordinal = chr(i+ord("A"))
        to_ret_parsed.append({
            "title": f'DOCUMENT {ordinal} - {parsed_content[0]["content"][0]}',
            "ordinal": ordinal,
            "content": parsed_content,
        })
        to_ret_raw.append(raw_content)
    return to_ret_parsed, to_ret_raw


In [159]:
policy = models.Policy.objects.get(site_name="amazon.com")
pi_content, raw_pi_content = create_policy_instance_content(parsed_docs["www.amazon.com"].items())
scan_dt = datetime.datetime.now()

raw_policy_instance = models.RawPolicyInstance(
    policy_id=policy.id,
    raw_content_blocks=raw_pi_content,
    capture_date=scan_dt,
    capture_source="chrome-addon"
)
raw_policy_instance.save()
policy_instance = models.PolicyInstance(
    policy_id=policy.id,
    raw_policy_id=raw_policy_instance.id,
    scan_dt=raw_policy_instance.capture_date,
    content=pi_content
)
policy_instance.save()